# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

os.getcwd()

'C:\\Users\\timos\\OneDrive\\Documents\\Springboard\\06-GuidedCapstone'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
df = pd.read_csv('data/step3_output.csv')
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,1,0.0,0,2,2,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,Arizona Snowbowl,Arizona,11500,2300,0,0.0,1,0,2,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


In [3]:
df.shape

(330, 26)

In [4]:
df.isnull().sum()

Name                 0
state                0
summit_elev          0
vertical_drop        0
trams                0
fastEight            0
fastSixes            0
fastQuads            0
quad                 0
triple               0
double               0
surface              0
total_chairs         0
Runs                 0
TerrainParks         0
LongestRun_mi        0
SkiableTerrain_ac    0
Snow Making_ac       0
daysOpenLastYear     0
yearsOpen            0
averageSnowfall      0
AdultWeekday         0
AdultWeekend         0
projectedDaysOpen    0
NightSkiing_ac       0
clusters             0
dtype: int64

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

Here I have decided to switch from the pd.get_dummies method to the sklearn OneHotEncoder.

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [5]:
# first we import the preprocessing package from the sklearn library
# adding make_column_transformer and OneHotEncoder
from sklearn import preprocessing
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
# scaler = preprocessing.StandardScaler().fit(X)
# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
# X_scaled=scaler.transform(X)

ohe = OneHotEncoder(handle_unknown='ignore')  
scaler = preprocessing.StandardScaler()

df2 = df.drop(['Name','AdultWeekend'], axis=1)
 
ct = make_column_transformer(
    (ohe, ['state', 'clusters']), 
    (scaler, ['summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac']), 
    remainder='passthrough')

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [6]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

X = df2
y = df.AdultWeekend

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(247, 24)
(83, 24)
(247,)
(83,)


Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [8]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

In [9]:
lm = linear_model.LinearRegression()

In [10]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(ct, lm)

In [11]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['state', 'clusters']),
                    

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [12]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = pipe.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [ ]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [13]:
print(explained_variance_score(y_test,y_pred))
print('\n')
print(mean_absolute_error(y_test,y_pred))

0.9420320973866998


4.885735603705143


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [16]:
lm.intercept_

-54375059282559.12

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [17]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
l = pipe.named_steps['columntransformer'].transformers_[0][1].categories_
l[1]
column_names = ['summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac'] + list(l[0]) + list(l[1])
len(column_names)

60

In [18]:
coeff_df = pd.DataFrame(abs(pipe.named_steps['linearregression'].coef_)
                        ,column_names, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
Idaho,5.437506e+13
NightSkiing_ac,5.437506e+13
Maryland,5.437506e+13
total_chairs,5.437506e+13
Runs,5.437506e+13
Iowa,5.437506e+13
yearsOpen,5.437506e+13
fastSixes,5.437506e+13
Colorado,5.437506e+13
California,5.437506e+13


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [19]:
X2=df.drop(['AdultWeekend','Name','state'], axis=1)
y=df.AdultWeekend
y=y.ravel()

In [20]:
ct2 = make_column_transformer(
    (ohe, ['clusters']),  
    (scaler, ['summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac']),
    remainder='passthrough')

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=1)

In [22]:
lm = linear_model.LinearRegression()

In [23]:
pipe2 = make_pipeline(ct2, lm)
pipe2.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['clusters']),
                             

In [25]:
y_pred = pipe2.predict(X_test)

In [26]:
print(explained_variance_score(y_test,y_pred))
print('\n')
print(mean_absolute_error(y_test,y_pred))

0.9334517348535325


5.243236239712175


In [27]:
lm.intercept_

69.68659625775886

In [29]:
l2 = pipe2.named_steps['columntransformer'].transformers_[0][1].categories_
column_names2 = ['summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac'] + list(l2[0])

In [32]:
coeff_df = pd.DataFrame(abs(pipe2.named_steps['linearregression'].coef_), column_names2, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
0,18.635834
trams,15.414367
summit_elev,13.323530
fastEight,4.250712
yearsOpen,2.435182
vertical_drop,2.090837
TerrainParks,1.400920
total_chairs,1.385166
fastSixes,1.192956
double,1.156226


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [33]:
X3=df.drop(['AdultWeekend','Name','state','summit_elev'], axis=1)
y=df.AdultWeekend
y=y.ravel()

In [34]:
ct3 = make_column_transformer(
    (ohe, ['clusters']),  
    (scaler, ['vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac']),
    remainder='passthrough')

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=1)

In [36]:
lm = linear_model.LinearRegression()

In [37]:
pipe3 = make_pipeline(ct3, lm)
pipe3.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['clusters']),
                             

In [38]:
y_pred = pipe3.predict(X_test)

In [39]:
print(explained_variance_score(y_test,y_pred))
print('\n')
print(mean_absolute_error(y_test,y_pred))

0.9313234692150453


5.341602455909205


In [40]:
lm.intercept_

80.69123152285475

In [41]:
l3 = pipe3.named_steps['columntransformer'].transformers_[0][1].categories_
column_names3 = ['vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'projectedDaysOpen',
       'NightSkiing_ac'] + list(l3[0])

In [42]:
coeff_df = pd.DataFrame(abs(pipe3.named_steps['linearregression'].coef_), 
                        column_names3, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
fastEight,35.641166
vertical_drop,20.141199
0,18.779927
trams,15.499967
yearsOpen,4.194303
fastSixes,2.169882
total_chairs,1.411540
AdultWeekday,1.405244
TerrainParks,1.259771
surface,1.106267


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [43]:
Comparison = pd.DataFrame({'Model':['Model 1', 'Model 2', 'Model 3'], 
                             'Explained Variance': [0.9420320973866998, 0.9334517348535325, 0.9313234692150453],
                             'Mean Absolute Error': [4.885735603705143, 5.243236239712175, 5.341602455909205],
                             'Features Dropped': ['-', 'states', 'states, summit_elev']})

In [44]:
Comparison

,Model,Explained Variance,Mean Absolute Error,Features Dropped
0,Model 1,0.942032,4.885736,-
1,Model 2,0.933452,5.243236,states
2,Model 3,0.931323,5.341602,"states, summit_elev"
